In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
gpath = '/content/gdrive/My Drive/MLdata/'
import os
os.path.exists(gpath)
# os.chdir(gpath)
#############End of colab code ###################
# Import the environment
print(os.listdir(gpath))
dpath = os.path.join(gpath, 'padded_4000_train_test.pkl')
print(os.path.exists(dpath))
tkpath = os.path.join(gpath, 'tk.pkl')
print(os.path.exists(tkpath))

Mounted at /content/gdrive
['padded_4000_train_test.pkl', 'tk.pkl']
True
True


In [8]:
def test_model(ablation=1000, B=32, epochs=3, filters=64, kernel_size=3, dense_neurons=2048, 
              conv1d_set1=1, conv1d_set2=1, maxpool_1=False):
    
    import tensorflow as tf
    import numpy as np
    tf.random.set_seed(123)
    import pickle
    with open(dpath, 'rb') as f:
      padded_4000_train_test = pickle.load(f)    
    x_train, y_train, x_test, y_test = padded_4000_train_test
    train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_data = train_data.shuffle(buffer_size=y_train.shape[0]).batch(B, drop_remainder=True)
    print(train_data)
    
    test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    test_data = test_data.shuffle(buffer_size=y_test.shape[0]).batch(B, drop_remainder=True)
    print(test_data)
    
    
    # tk = hdfslogs.tk  
    with open(tkpath, 'rb') as f:
      tk = pickle.load(f)  
    vocab_size = len(tk.word_index)
    vocab_size = len(tk.word_index)
    print(f'vocab_size: {vocab_size}')
    char_onehot = vocab_size
    
    embedding_weights = []
    embedding_weights.append(np.zeros(vocab_size))
    for char, i in tk.word_index.items(): # from 1 to 51
        onehot = np.zeros(vocab_size)
        onehot[i-1] = 1
        embedding_weights.append(onehot)
    embedding_weights = np.array(embedding_weights)
    
    input_size = [x_train.shape[1], x_train.shape[2]]
    embedding_size = vocab_size

    embedding_layer = tf.keras.layers.Embedding(vocab_size+1,
                                                embedding_size,
                                                input_length=input_size,
                                                weights = [embedding_weights])
    
    
    
    
    inputs = tf.keras.layers.Input(batch_shape=(B, x_train.shape[1], x_train.shape[2]), dtype='float64' )
    x = tf.keras.layers.Embedding(input_dim=vocab_size+1,
                                    output_dim=embedding_size,
                                    input_length=x_train.shape[2],
                                    weights = [embedding_weights],
                                    )(inputs)
    for _ in range(conv1d_set1):
        x = tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    if maxpool_1:
        x = tf.keras.layers.MaxPooling2D(pool_size=(1, x_train.shape[2]))(x)
        x = tf.reshape(x, (B, x_train.shape[1], filters))        
        for _ in range(conv1d_set2):
            x = tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
        x = tf.keras.layers.MaxPooling1D(pool_size=(x_train.shape[1]) )(x)
        x = tf.reshape(x, (B, filters))
    if not maxpool_1:
        x = tf.keras.layers.Flatten()(x)       
    x = tf.keras.layers.Dense(dense_neurons)(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    print(model.summary())
    model.compile(optimizer='adam', 
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    hist = model.fit(train_data, validation_data=test_data, epochs=epochs) 
    return model, hist

In [9]:
test_model(B=250, kernel_size=3, epochs=16, dense_neurons=2048, conv1d_set1=3,conv1d_set2=3, maxpool_1=True )

<BatchDataset shapes: ((250, 64, 256), (250,)), types: (tf.int32, tf.int64)>
<BatchDataset shapes: ((250, 64, 256), (250,)), types: (tf.int32, tf.int64)>
vocab_size: 42
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(250, 64, 256)]          0         
                                                                 
 embedding_5 (Embedding)     (250, 64, 256, 42)        1806      
                                                                 
 conv1d_12 (Conv1D)          (250, 64, 256, 64)        8128      
                                                                 
 conv1d_13 (Conv1D)          (250, 64, 256, 64)        12352     
                                                                 
 conv1d_14 (Conv1D)          (250, 64, 256, 64)        12352     
                                                                 
 max_pooling2d_2 (MaxP

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


32/32 [==============================] - 28s 551ms/step - loss: 0.4336 - accuracy: 0.7859 - precision_1: 0.8498 - recall: 0.6945 - val_loss: 0.3015 - val_accuracy: 0.9128 - val_precision_1: 0.8534 - val_recall: 0.9967
Epoch 2/16
32/32 [==============================] - 17s 527ms/step - loss: 0.0612 - accuracy: 0.9805 - precision_1: 0.9868 - recall: 0.9740 - val_loss: 0.0165 - val_accuracy: 0.9971 - val_precision_1: 0.9948 - val_recall: 0.9995
Epoch 3/16
32/32 [==============================] - 17s 533ms/step - loss: 0.0095 - accuracy: 0.9979 - precision_1: 0.9975 - recall: 0.9983 - val_loss: 0.0076 - val_accuracy: 0.9980 - val_precision_1: 0.9960 - val_recall: 1.0000
Epoch 4/16
32/32 [==============================] - 17s 528ms/step - loss: 0.0033 - accuracy: 0.9991 - precision_1: 0.9983 - recall: 1.0000 - val_loss: 0.0061 - val_accuracy: 0.9985 - val_precision_1: 0.9970 - val_recall: 1.0000
Epoch 5/16
32/32 [==============================] - 17s 531ms/step - loss: 0.0021 - accuracy: 0

(<keras.engine.functional.Functional at 0x7f0eeecbbe90>,
 <keras.callbacks.History at 0x7f0eeed27650>)